# TitaSink - Random Tree Classifier

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
cwd = os.getcwd()

In [3]:
path = os.path.join(cwd,'train.csv')
print(path)
data = pd.read_csv(path)

C:\Users\User\Documents\GitHub\TitaSink\train.csv


In [4]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
def preprocess(data, onehot=True):
    
    data['Sex'].replace(['female','male'],[0,1], inplace=True) 
    # l'attribut Sex est DANS CE DATASET binaire. 
    # On peut donc facilement remplacer les valeurs de cet attribut par des valeurs binaires.
    # Noter que l'argument inplace de la fonction permet d'avoir une transformation sous forme 
    # de procédure plutôt que sous forme de fonction. Cela permet d'éviter le recopiage mémoire du dataset.
    
    data.drop(columns=['Name', 'Cabin', 'Ticket','PassengerId'], inplace=True) 
    # On supprime du dataset les colonnes qui ne présentent pas d'intérêt pour la classification 
    # L'attribut Cabin présente trop de missing value NaN
    # L'attribut Ticket est assez indéchiffrable 
    # Le Nom ne présente pas d'intérêt pour la complexité de cette étude. Peut-être qu'une étude NLP pourrait 
    # montrer des dépendances entre le nom et la survivabilité du passager en fonction de son origine (réaction 
    # différente face au danger en fonction des origines ? ). 
    
    data['Embarked'].fillna('C', inplace=True) 
    # L'attribut Embarked a 2 missing values. On peut penser que les remplir aléatoirement est une bonne idée 
    # dans une première approximation sans avoir d'autres idées sur les dépendances inter-attributs. Est-ce que cette 
    # technique peut quand même générer des outliers si des dépendances existent entre les attributs ? 
    
    index = data[data['Age'].isna()].index # REALLY IMPORTANT
    value = np.absolute(np.random.normal(loc=data['Age'].mean(), scale=data['Age'].std(), size=data['Age'].isna().sum()))
    data['Age'].fillna(pd.Series(value, index=index), inplace=True)    
    # L'attribut Age présente beaucoup de missing values. Sans information supplémentaire, remplir les valeurs 
    # manquantes avec une loi normale est une solution simple et efficace. 
    # https://stackoverflow.com/questions/53104625/pandas-fill-in-missing-values-choosing-values-from-a-normal-distribution
    # Cependant, compte tenu du nombre important de missing value, peut-être qu'avoir des informations avant 
    # d'inputer les missing values serait bénéfique pour ne pas influer trop sur l'information contenue dans 
    # le dataset par l'attribut Age. 
    
    if onehot:
        data = pd.get_dummies(data, columns=['Pclass', 'Embarked'])
        # Enfin, les variables categorical se doivent d'être one-hot encoded pour conserver l'information de classes 
        # sous forme numérique. Chaque valeur possible l'attribut apporte une colonne binaire supplémentaire dans le 
        # dataset final. 
    else:
        True
    
    return data

In [7]:
data_train = data.copy()

n=80
X_train = data_train.drop('Survived', axis=1).head(int(len(data)*n/100))
X_train = preprocess(X_train)
Y_train = data_train['Survived'].head(int(len(data)*n/100))

X_val = data_train.drop('Survived', axis=1).tail(int(len(data)*(100-n)/100)+1)
X_val = preprocess(X_val)
Y_val = data_train['Survived'].tail(int(len(data)*(100-n)/100)+1)

In [8]:
scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
# Fit to data, then transform it.
X_val = scaler.transform(X_val)
# Transform the data with previous statistical parameters
# https://datascience.stackexchange.com/questions/38395/standardscaler-before-and-after-splitting-data

In [9]:
ForestClf = RandomForestClassifier(n_estimators=100, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
ForestClf.fit(X_train, Y_train)
ForestClf.score(X_train, Y_train)

0.8960674157303371

## Prediction for the test file

In [10]:
X_TOT = data.drop('Survived', axis=1)
X_TOT = preprocess(X_TOT)
Y_TOT = data['Survived']

In [11]:
scaler_test=StandardScaler()
X_TOT = scaler_test.fit_transform(X_TOT)
# Fit to data, then transform it.

In [12]:
ForestClf = RandomForestClassifier(n_estimators=100, min_samples_leaf=3, max_features=0.5, n_jobs=-1)
ForestClf.fit(X_TOT, Y_TOT)
ForestClf.score(X_TOT, Y_TOT)

0.9079685746352413

In [13]:
path = os.path.join(cwd,'test.csv')
print(path)
data_test = pd.read_csv(path)

C:\Users\User\Documents\GitHub\TitaSink\test.csv


In [14]:
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [15]:
X_test = preprocess(data_test)
X_test.head()

,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,1,34.5,0,0,7.8292,0,0,1,0,1,0
1,0,47.0,1,0,7.0000,0,0,1,0,0,1
2,1,62.0,0,0,9.6875,0,1,0,0,1,0
3,1,27.0,0,0,8.6625,0,0,1,0,0,1
4,0,22.0,1,1,12.2875,0,0,1,0,0,1


In [16]:
X_test.isna().sum()

Sex           0
Age           0
SibSp         0
Parch         0
Fare          1
Pclass_1      0
Pclass_2      0
Pclass_3      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [17]:
X_test["Fare"].fillna(method='backfill', inplace=True)

In [18]:
X_test = scaler_test.transform(X_test)
predict = ForestClf.predict(X_test)

In [19]:
np.size(predict)

418

In [20]:
import csv
with open('submission_RDC.csv', 'w', newline='') as csvfile:
    Pred_writer = csv.writer(csvfile, delimiter=',')
    Pred_writer.writerow(['PassengerId','Survived'])
    for i in range(len(predict)):
        Pred_writer.writerow([892+i, predict[i]])